# Web scraping de literales con Python

In [1]:
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os

import time

import numpy as np
import pandas as pd

In [2]:
# Instantiate options
opts = Options()

# opts.add_argument(" — headless") # Uncomment if the headless version needed
opts.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set the location of the webdriver
chrome_driver = os.getcwd() + "\chromedriver.exe"



In [6]:
sitemap = "https://www.mediamarkt.es/es/legal/site-map"

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
# Load the HTML page
driver.maximize_window()
driver.get(sitemap)
#driver.implicitly_wait(220)
#time.sleep(10)
#javaScript = "window.scrollTo(0,document.body.scrollHeight);"
#driver.execute_script(javaScript)
#time.sleep(5)
soup = BeautifulSoup(driver.page_source)
driver.close()

In [7]:
# Find list of main links
sections = soup.find_all('div', {"data-test":"foldable"})
main_links = list(map(lambda x: x.find_all('a')[0]['href'], sections))[:-1] #El último es ofertas, no nos interesa
main_links

['https://www.mediamarkt.es/es/category/_informática-701127.html',
 'https://www.mediamarkt.es/es/category/_telefonía-701128.html',
 'https://www.mediamarkt.es/es/category/_televisión-701130.html',
 'https://www.mediamarkt.es/es/category/_audio-y-hifi-701131.html',
 'https://www.mediamarkt.es/es/category/_smart-home-701137.html',
 'https://www.mediamarkt.es/es/category/_consolas-y-videojuegos-701147.html',
 'https://www.mediamarkt.es/es/category/_electrodomésticos-701132.html',
 'https://www.mediamarkt.es/es/category/_pequeños-electrodomésticos-778049.html',
 'https://www.mediamarkt.es/es/category/_belleza-y-salud-701113.html',
 'https://www.mediamarkt.es/es/category/_climatización-y-calefacción-701134.html',
 'https://www.mediamarkt.es/es/category/_deporte-y-movilidad-701146.html',
 'https://www.mediamarkt.es/es/category/_fotografía-701129.html',
 'https://www.mediamarkt.es/es/category/_cine-música-y-libros-701148.html']

In [26]:
# Find links for all categories for a given main link
def cat_links(main_link):
    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    # Load the HTML page
    driver.maximize_window()
    driver.get(main_link)
    time.sleep(2)
    # Click en Aceptar cookies
    driver.find_element_by_xpath('//*[@id="privacy-layer-accept-all-button"]').click()
    # Click en Mostrar más (si existe):
    try:
        driver.find_elements(By.XPATH,'//button[text()="Mostrar más"]')[0].click()
    except:
        pass
    # Obtener todos los links de las categorías
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    cats = soup.find_all('div',{'class':'CategoryContentSidenavItem__Wrapper-sc-1oj9bnf-2 blRnEW'})[0].find_all('a')
    names = []
    links = []
    for cat in cats:
        names.append(cat.find('span').getText())
        links.append(cat['href'])
    return list(zip(names,links))

#cat_links('https://www.mediamarkt.es/es/category/_deporte-y-movilidad-701146.html')

[('Patinetes eléctricos',
  'https://www.mediamarkt.es/es/category/patinetes-el%C3%A9ctricos-820.html'),
 ('Bicicletas eléctricas',
  'https://www.mediamarkt.es/es/category/bicicletas-el%C3%A9ctricas-817.html'),
 ('Hoverboards y segways',
  'https://www.mediamarkt.es/es/category/hoverboards-y-segways-811.html'),
 ('Accesorios movilidad urbana',
  'https://www.mediamarkt.es/es/category/accesorios-movilidad-urbana-824.html')]

In [27]:
# Find all links to products
prod_links = []
for link in main_links:
    prod_links += cat_links(link)

prod_links

[('Portátiles',
  'https://www.mediamarkt.es/es/category/port%C3%A1tiles-153.html'),
 ('PC sobremesa',
  'https://www.mediamarkt.es/es/category/pc-sobremesa-161.html'),
 ('Monitores', 'https://www.mediamarkt.es/es/category/monitores-179.html'),
 ('Tablets', 'https://www.mediamarkt.es/es/category/tablets-169.html'),
 ('Impresoras', 'https://www.mediamarkt.es/es/category/impresoras-226.html'),
 ('eBooks', 'https://www.mediamarkt.es/es/category/ebooks-249.html'),
 ('Scanners', 'https://www.mediamarkt.es/es/category/scanners-248.html'),
 ('Software', 'https://www.mediamarkt.es/es/category/software-257.html'),
 ('Smartphones', 'https://www.mediamarkt.es/es/category/smartphones-263.html'),
 ('Smartwatches',
  'https://www.mediamarkt.es/es/category/smartwatches-265.html'),
 ('Accesorios para móviles',
  'https://www.mediamarkt.es/es/category/accesorios-para-m%C3%B3viles-274.html'),
 ('Móviles básicos',
  'https://www.mediamarkt.es/es/category/m%C3%B3viles-b%C3%A1sicos-264.html'),
 ('Telefonía

In [19]:
cats = soup.find_all('div',{'class':'CategoryContentSidenavItem__Wrapper-sc-1oj9bnf-2 blRnEW'})[0].find_all('a')
cats

[<a class="Linkstyled__StyledLinkAnchor-sc-1drhx1h-0 dbmmtY CategoryContentSidenavItem__StyledLink-sc-1oj9bnf-1 grZYpe" href="https://www.mediamarkt.es/es/category/port%C3%A1tiles-153.html" target="_self"><span class="Typostyled__StyledInfoTypo-sc-1jga2g7-0 gQDatN Linkstyled__Text-sc-1drhx1h-4 kTEKRg" color="#272422" font-size="xs">Portátiles</span></a>,
 <a class="Linkstyled__StyledLinkAnchor-sc-1drhx1h-0 dbmmtY CategoryContentSidenavItem__StyledLink-sc-1oj9bnf-1 grZYpe" href="https://www.mediamarkt.es/es/category/pc-sobremesa-161.html" target="_self"><span class="Typostyled__StyledInfoTypo-sc-1jga2g7-0 gQDatN Linkstyled__Text-sc-1drhx1h-4 kTEKRg" color="#272422" font-size="xs">PC sobremesa</span></a>,
 <a class="Linkstyled__StyledLinkAnchor-sc-1drhx1h-0 dbmmtY CategoryContentSidenavItem__StyledLink-sc-1oj9bnf-1 grZYpe" href="https://www.mediamarkt.es/es/category/monitores-179.html" target="_self"><span class="Typostyled__StyledInfoTypo-sc-1jga2g7-0 gQDatN Linkstyled__Text-sc-1drhx1h-

In [28]:
def get_names(web):
    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    # Load the HTML page
    driver.maximize_window()
    driver.get(web)
    time.sleep(2)
    #Click en Aceptar cookies
    driver.find_element_by_xpath('//*[@id="privacy-layer-accept-all-button"]').click()
    
    #Click en Mostrar más productos (si existe)
    try:
        driver.find_element_by_xpath('//button[@data-test="mms-search-srp-loadmore"]').click()
    except:
        pass
    
    #Scroll infinito
    scroll_pause_time = 0.7 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 2

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 2
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break
            
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    items = soup.find_all('p',{'data-test':'product-title'})
    return list(map(lambda x: x.getText(), items))

#web = 'https://www.mediamarkt.es/es/category/heladeras-657.html'
#web = 'https://www.mediamarkt.es/es/category/aspiradores-y-limpieza-1034.html'
#literales = get_names(web)

In [33]:
# Scrape all links
data = pd.DataFrame(columns = ['Categoría','Literal'])
for cat,link in prod_links:
    literales = get_names(link)
    link_data = pd.DataFrame({'Categoría':cat,'Literal':literales})
    data = data.append(link_data)

In [34]:
len(data)

15616

In [35]:
data.to_excel('literales_mediamarkt.xlsx')